# Brazilian E-Commerce Public Dataset by Olist

## About this file

This dataset contains detailed information about customers and their locations. It is primarily used to identify unique customers within the orders dataset and to determine the delivery location of each order.

In the Olist system, every order is assigned a distinct customer_id. As a result, the same customer may appear with different customer_ids across multiple orders. To address this, the dataset also includes a customer_unique_id field, which enables you to identify customers who have made repeat purchases. Without customer_unique_id, it would appear as though each order were linked to a completely different customer.

## Context

This dataset has been generously provided by Olist, the largest department store on Brazilian marketplaces. Olist connects small businesses from across Brazil to online sales channels seamlessly through a single contract. These merchants sell products via the Olist Store and ship directly to customers using Olist’s logistics partners.
Learn more at: www.olist.com

After a customer purchases a product from the Olist Store, the seller is notified to fulfill the order. Once the customer receives the product—or the estimated delivery date passes—the customer receives a satisfaction survey via email to rate their experience and provide feedback.

## Acknowledgements
Special thanks to Olist for making this dataset publicly available.

## QUESTION
Use appropriate machine learning model to demonstrate how machine learning can be used on e-commerce sites for product recommendations to users.

# Imports

In [1]:
# load imports
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import os

# Download the datasets from kaggle

In [2]:
# Kaggle dataset download link
url = "https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce"

cwd=os.getcwd()

# Destination folder
if 'brazilian-ecommerce' in os.listdir():
    print('The folder already exist')
    extract_dir = os.path.join(cwd, 'brazilian-ecommerce')
else:
    # Download file
    response = requests.get(url)

    # Unzip directly from memory
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(extract_dir)

The folder already exist


In [3]:
# check dataset names
datasets=os.listdir(extract_dir)
for name in datasets:
    print(name)

olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv


In [4]:
# create function to open all the datasets
def open_csv(file_no):
    lst=[]
    for name in datasets:
        lst.append(name)
    df=pd.read_csv('brazilian-ecommerce/'+lst[file_no])
    return df

# Inspecting the datasets (EDA)

### 1. customers_dataset

In [5]:
customers_dataset=open_csv(0)
customers_dataset.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [6]:
# inspect nulls
customers_dataset.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [7]:
# inspect duplicates
customers_dataset[customers_dataset.duplicated()]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state


In [8]:
# inspect data types
customers_dataset.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [9]:
# inspect categories
print(customers_dataset['customer_city'].unique())
print(customers_dataset['customer_state'].unique())

['franca' 'sao bernardo do campo' 'sao paulo' ... 'monte bonito'
 'sao rafael' 'eugenio de castro']
['SP' 'SC' 'MG' 'PR' 'RJ' 'RS' 'PA' 'GO' 'ES' 'BA' 'MA' 'MS' 'CE' 'DF'
 'RN' 'PE' 'MT' 'AM' 'AP' 'AL' 'RO' 'PB' 'TO' 'PI' 'AC' 'SE' 'RR']


In [10]:
# shape of the data
customers_dataset.shape

(99441, 5)

### 2. geolocation_dataset

In [11]:
geolocation_dataset=open_csv(1)
geolocation_dataset.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


- might be unnecessary for my question 

### 3. orders_dataset

In [12]:
orders_dataset=open_csv(2)
orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [13]:
# inspect nulls
orders_dataset.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [14]:
# inspect duplicated rows
orders_dataset[orders_dataset.duplicated()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [15]:
# inspect data types
orders_dataset.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [16]:
# inspect categories
orders_dataset['order_status'].value_counts()#unique()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

In [17]:
# drop orders whose status are 'unavailable' suggests stockout or system issue
available_orders =orders_dataset.loc[orders_dataset['order_status'] != 'unavailable']
available_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [18]:
# Delivered before purchase
anomaly_delivered_before_purchase = available_orders[
    available_orders['order_purchase_timestamp'] > available_orders['order_delivered_customer_date']
]
anomaly_delivered_before_purchase

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [19]:
# Carrier pickup before purchase 
anomaly_carrier_picup_before_purchase = available_orders[
    available_orders['order_purchase_timestamp'] > available_orders['order_delivered_carrier_date']
]
anomaly_carrier_picup_before_purchase[['order_purchase_timestamp', 'order_delivered_carrier_date']]

,order_purchase_timestamp,order_delivered_carrier_date
615,2018-08-16 13:50:48,2018-08-16 13:27:00
1111,2018-06-15 15:41:22,2018-06-15 14:52:00
1329,2018-05-02 10:48:44,2018-05-02 09:49:00
1372,2018-05-14 15:18:23,2018-05-14 13:46:00
1864,2018-07-26 13:25:14,2018-07-26 12:42:00
...,...,...
98172,2018-07-30 15:22:15,2018-07-30 15:00:00
98430,2018-05-04 14:50:37,2018-05-04 14:48:00
98672,2018-07-18 08:46:52,2018-07-18 08:44:00
98780,2018-07-18 16:14:16,2018-07-18 15:34:00


In [20]:
# Approval before purchase
anomaly_approval_before_purchase = available_orders[
    available_orders['order_approved_at'] < available_orders['order_purchase_timestamp']
]
anomaly_approval_before_purchase

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [21]:
# Carrier before approval
anomaly_carrier_before_approval = available_orders[
    available_orders['order_delivered_carrier_date'] < available_orders['order_approved_at']
]
anomaly_carrier_before_approval[['order_delivered_carrier_date','order_approved_at']]

,order_delivered_carrier_date,order_approved_at
15,2018-06-11 14:54:00,2018-06-12 23:31:02
64,2018-04-23 19:19:14,2018-04-24 18:25:22
199,2018-07-24 12:57:00,2018-07-26 23:31:53
210,2018-07-23 12:24:00,2018-07-23 12:31:53
415,2018-07-24 14:03:00,2018-07-27 23:31:09
...,...,...
99091,2018-07-05 14:11:00,2018-07-05 16:17:59
99230,2018-07-03 12:57:00,2018-07-05 16:32:52
99266,2018-01-31 18:11:58,2018-02-04 23:31:46
99377,2018-04-23 17:18:40,2018-04-24 19:26:10


In [22]:
# Customer delivery before carrier pickup
anomaly_delivery_before_carrier = available_orders[
    available_orders['order_delivered_customer_date'] < available_orders['order_delivered_carrier_date']
]
anomaly_delivery_before_carrier[['order_delivered_customer_date', 'order_delivered_carrier_date']]

,order_delivered_customer_date,order_delivered_carrier_date
6437,2017-07-25 19:32:56,2017-07-28 16:57:58
9553,2017-07-06 14:27:51,2017-07-07 17:22:41
13487,2017-07-19 14:13:28,2017-07-20 19:22:02
14474,2017-07-26 18:09:10,2017-08-01 18:23:30
19268,2017-07-20 18:52:41,2017-07-20 23:03:42
21338,2016-10-25 17:51:46,2016-10-26 11:41:53
22520,2017-06-26 15:45:35,2017-06-27 14:51:54
25393,2017-08-10 18:05:38,2017-08-10 18:28:56
25646,2017-07-31 17:49:56,2017-08-01 18:17:47
27470,2017-08-01 21:13:01,2017-08-09 18:18:43


In [23]:
# Delivery after estimated date
anomaly_late_delivery = available_orders[
    available_orders['order_delivered_customer_date'] > available_orders['order_estimated_delivery_date']
]
anomaly_late_delivery

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
20,203096f03d82e0dffbc41ebc2e2bcfb7,d2b091571da224a1b36412c18bc3bbfe,delivered,2017-09-18 14:31:30,2017-09-19 04:04:09,2017-10-06 17:50:03,2017-10-09 22:23:46,2017-09-28 00:00:00
25,fbf9ac61453ac646ce8ad9783d7d0af6,3a874b4d4c4b6543206ff5d89287f0c3,delivered,2018-02-20 23:46:53,2018-02-22 02:30:46,2018-02-26 22:25:22,2018-03-21 22:03:54,2018-03-12 00:00:00
35,8563039e855156e48fccee4d611a3196,5f16605299d698660e0606f7eae2d2f9,delivered,2018-02-17 15:59:46,2018-02-17 16:15:34,2018-02-20 23:03:56,2018-03-20 00:59:25,2018-03-20 00:00:00
41,6ea2f835b4556291ffdc53fa0b3b95e8,c7340080e394356141681bd4c9b8fe31,delivered,2017-11-24 21:27:48,2017-11-25 00:21:09,2017-12-13 21:14:05,2017-12-28 18:59:23,2017-12-21 00:00:00
57,66e4624ae69e7dc89bd50222b59f581f,684fa6da5134b9e4dab731e00011712d,delivered,2018-03-09 14:50:15,2018-03-09 15:40:39,2018-03-15 00:31:19,2018-04-03 13:28:46,2018-04-02 00:00:00
...,...,...,...,...,...,...,...,...
99396,b3112ca67f3afd4e20cf2ee91fc4f804,6f83c71b6c044fb156d697d4130fe9b5,delivered,2018-08-02 22:46:54,2018-08-02 23:04:06,2018-08-15 17:42:00,2018-08-21 00:03:26,2018-08-16 00:00:00
99397,0fa1fab1d7c1211c824596ed5e111e3c,7f3bd6c94d2daf7b6462d1a894a775b4,delivered,2018-03-13 21:48:57,2018-03-13 22:40:28,2018-03-14 19:27:23,2018-04-05 19:59:49,2018-04-02 00:00:00
99423,38e9133ce29f6bbe35aed9c3863dce01,ad312389a098ceff46ce92c4595c06d0,delivered,2017-10-12 20:54:11,2017-10-14 03:28:24,2017-10-17 17:04:42,2017-11-21 17:06:59,2017-10-31 00:00:00
99425,d692ef54145c9cb3322ec2e5508aa3f4,82ddfcf9438b0cd1117b55ac33184df8,delivered,2018-03-21 19:47:18,2018-03-21 20:05:26,2018-03-22 21:11:58,2018-04-11 00:48:31,2018-04-09 00:00:00


Drop the following:
- anomaly_delivery_before_carrier
- anomaly_carrier_before_approval
- anomaly_carrier_picup_before_purchase

In [24]:
anomalies=available_orders[
    (available_orders['order_purchase_timestamp'] > available_orders['order_delivered_carrier_date']) | 
    (available_orders['order_delivered_carrier_date'] < available_orders['order_approved_at']) | 
    (available_orders['order_delivered_customer_date'] < available_orders['order_delivered_carrier_date'])
]
anomalies

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
15,dcb36b511fcac050b97cd5c05de84dc3,3b6828a50ffe546942b7a473d70ac0fc,delivered,2018-06-07 19:03:12,2018-06-12 23:31:02,2018-06-11 14:54:00,2018-06-21 15:34:32,2018-07-04 00:00:00
64,688052146432ef8253587b930b01a06d,81e08b08e5ed4472008030d70327c71f,delivered,2018-04-22 08:48:13,2018-04-24 18:25:22,2018-04-23 19:19:14,2018-04-24 19:31:58,2018-05-15 00:00:00
199,58d4c4747ee059eeeb865b349b41f53a,1755fad7863475346bc6c3773fe055d3,delivered,2018-07-21 12:49:32,2018-07-26 23:31:53,2018-07-24 12:57:00,2018-07-25 23:58:19,2018-07-31 00:00:00
210,412fccb2b44a99b36714bca3fef8ad7b,c6865c523687cb3f235aa599afef1710,delivered,2018-07-22 22:30:05,2018-07-23 12:31:53,2018-07-23 12:24:00,2018-07-24 19:26:42,2018-07-31 00:00:00
415,56a4ac10a4a8f2ba7693523bb439eede,78438ba6ace7d2cb023dbbc81b083562,delivered,2018-07-22 13:04:47,2018-07-27 23:31:09,2018-07-24 14:03:00,2018-07-28 00:05:39,2018-08-06 00:00:00
...,...,...,...,...,...,...,...,...
99091,240ead1a7284667e0ec71d01f80e4d5e,fcdd7556401aaa1c980f8b67a69f95dc,delivered,2018-07-02 16:30:02,2018-07-05 16:17:59,2018-07-05 14:11:00,2018-07-10 23:21:47,2018-07-24 00:00:00
99230,78008d03bd8ef7fcf1568728b316553c,043e3254e68daf7256bda1c9c03c2286,delivered,2018-07-03 13:11:13,2018-07-05 16:32:52,2018-07-03 12:57:00,2018-07-10 17:47:39,2018-07-23 00:00:00
99266,76a948cd55bf22799753720d4545dd2d,3f20a07b28aa252d0502fe7f7eb030a9,delivered,2018-01-30 02:41:30,2018-02-04 23:31:46,2018-01-31 18:11:58,2018-03-18 20:08:50,2018-03-02 00:00:00
99377,a6bd1f93b7ff72cc348ca07f38ec4bee,6d63fa86bd2f62908ad328325799152f,delivered,2018-04-20 17:28:40,2018-04-24 19:26:10,2018-04-23 17:18:40,2018-04-28 17:38:42,2018-05-15 00:00:00


In [25]:
# Remove all the anomalies
clean_orders = available_orders.drop(anomalies.index).copy().dropna()
clean_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


In [26]:
# ckeck missing values
clean_orders.isna().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

In [27]:
# select the columns suited for thequestion
clean_orders=clean_orders[['order_id', 'customer_id', 'order_status']].drop_duplicates()
clean_orders

,order_id,customer_id,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered
...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered


### 4. ordered_items_dataset

In [28]:
order_items_dataset=open_csv(3)
order_items_dataset.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [29]:
# inspect missing values
order_items_dataset.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [30]:
# inspect duplicates
order_items_dataset[order_items_dataset.duplicated()]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value


In [31]:
# inspect data types
order_items_dataset.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [32]:
# inspect data validity
order_items_dataset[['price', 'freight_value']].describe() #'shipping_limit_date' is a float

,price,freight_value
count,112650.000000,112650.000000
mean,120.653739,19.990320
std,183.633928,15.806405
min,0.850000,0.000000
25%,39.900000,13.080000
50%,74.990000,16.260000
75%,134.900000,21.150000
max,6735.000000,409.680000


In [33]:
# add price to freight_value as product price
order_items_dataset['product_price']=order_items_dataset['price']+order_items_dataset['freight_value']
order_items_dataset

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,218.04
...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,343.40
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,386.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,116.85
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,64.71


In [34]:
# select only the columns of interest
order_items_dataset=order_items_dataset[['order_id', 'order_item_id', 'product_id', 'product_price']].drop_duplicates()
order_items_dataset

,order_id,order_item_id,product_id,product_price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,218.04
...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,343.40
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,386.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,116.85
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,64.71


### 5. order_payments_dataset

In [35]:
order_payments_dataset=open_csv(4)
order_payments_dataset.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [36]:
# inspect missing values
order_payments_dataset.isna().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [37]:
# inspect duplicate values
order_payments_dataset[order_payments_dataset.duplicated()]

,order_id,payment_sequential,payment_type,payment_installments,payment_value


In [38]:
# inspect data types
order_payments_dataset.dtypes

order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

In [39]:
# inspect data validity
order_payments_dataset['payment_type'].value_counts()

payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64

In [40]:
order_payments_dataset[['payment_sequential', 'payment_installments', 'payment_value']].describe()

,payment_sequential,payment_installments,payment_value
count,103886.000000,103886.000000,103886.000000
mean,1.092679,2.853349,154.100380
std,0.706584,2.687051,217.494064
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,56.790000
50%,1.000000,1.000000,100.000000
75%,1.000000,4.000000,171.837500
max,29.000000,24.000000,13664.080000


In [41]:
seq_list=[]
for i in order_payments_dataset['payment_sequential'].unique():
    seq_list.append(i)
np.array(sorted(seq_list))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int64)

In [42]:
ins_list=[]
for j in order_payments_dataset['payment_installments'].unique():
    ins_list.append(j)
np.array(sorted(ins_list))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 20, 21, 22, 23, 24], dtype=int64)

### 6. order_reviews_dataset

In [43]:
order_reviews_dataset=open_csv(5)
order_reviews_dataset.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [44]:
# inspect missing values
order_reviews_dataset.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [45]:
# inspect dimensionality
order_reviews_dataset.shape

(99224, 7)

In [46]:
# inspect duplicate values
order_reviews_dataset[order_reviews_dataset.duplicated()]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp


In [47]:
# inspect validity
order_reviews_dataset['review_score'].unique()

array([4, 5, 1, 3, 2], dtype=int64)

In [48]:
r_comments=[review for review in order_reviews_dataset['review_comment_message'].unique()]
pd.Series(r_comments).sample(10, random_state=42)

12592    Paguei dois produtos com 60 e só veio um na no...
31156                       produto de acordo com o preço.
34481    Nenhum dos produtos que comprei desta loja vie...
22518                 Adorei foi entregue antes do prazo !
5503     Não veio na cor conforme o anuncio, a mochila ...
8413     Muito bom o prazo de entrega veio que eu esper...
14284                porem não recebi meus produtos Ainda 
7495         Oi....veio certinho e muuuuuito rápido...amei
33586                 Bem diferente do que aparece na foto
22456    o produto foi entregue antes do prazo , produt...
dtype: object

In [49]:
# drop columns review_comment_title	and review_comment_message
order_reviews_dataset.drop(columns=['review_comment_title',	'review_comment_message'], inplace=True)
order_reviews_dataset.isna().sum()

review_id                  0
order_id                   0
review_score               0
review_creation_date       0
review_answer_timestamp    0
dtype: int64

In [50]:
order_reviews_dataset[order_reviews_dataset.duplicated()]

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp


In [51]:
order_reviews_dataset=order_reviews_dataset[['review_id', 'order_id', 'review_score']].drop_duplicates()
order_reviews_dataset

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5
...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4


### 7. products_dataset

In [52]:
products_dataset=open_csv(6)
products_dataset.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [53]:
# check columns
products_dataset.columns

Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

In [54]:
# select columns of interes by dropping product_name_lenght, product_description_length and product_photos_qty
products_dataset.drop(columns=['product_name_lenght', 'product_description_lenght', 'product_photos_qty'], inplace=True)
products_dataset.isna().sum()/products_dataset.shape[0]*100

product_id               0.000000
product_category_name    1.851234
product_weight_g         0.006070
product_length_cm        0.006070
product_height_cm        0.006070
product_width_cm         0.006070
dtype: float64

In [55]:
products_dataset.shape

(32951, 6)

In [56]:
# select columns
products_dataset=products_dataset[['product_id', 'product_category_name', 'product_weight_g', 'product_length_cm', \
                                   'product_height_cm', 'product_width_cm']].drop_duplicates()
products_dataset

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,700.0,31.0,13.0,20.0


In [57]:
# inspect missing values
products_dataset.dropna(inplace=True)

In [58]:
round((products_dataset.isna().sum()/products_dataset.shape[0])*100,2)

product_id               0.0
product_category_name    0.0
product_weight_g         0.0
product_length_cm        0.0
product_height_cm        0.0
product_width_cm         0.0
dtype: float64

In [59]:
# inspect duplicate values
products_dataset[products_dataset.duplicated()]

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm


In [60]:
# inspect categories
list(pd.Series(products_dataset['product_category_name'].sample(10, random_state=2).unique()).sort_values())

['beleza_saude',
 'brinquedos',
 'cama_mesa_banho',
 'climatizacao',
 'eletrodomesticos',
 'pet_shop',
 'utilidades_domesticas']

In [61]:
products_dataset.describe()

,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32340.000000,32340.000000,32340.000000,32340.000000
mean,2276.956586,30.854545,16.958813,23.208596
std,4279.291845,16.955965,13.636115,12.078762
min,0.000000,7.000000,2.000000,6.000000
25%,300.000000,18.000000,8.000000,15.000000
50%,700.000000,25.000000,13.000000,20.000000
75%,1900.000000,38.000000,21.000000,30.000000
max,40425.000000,105.000000,105.000000,118.000000


### 8. sellers_dataset

In [62]:
sellers_dataset=open_csv(7)
sellers_dataset.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


- the seller might not be of interest for the problem

### 9. category_name_translation

In [63]:
category_name_translation=open_csv(8)
category_name_translation

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [64]:
# inspect missing values
category_name_translation.isna().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

In [65]:
# inspect duplicate values
category_name_translation[category_name_translation.duplicated()]

,product_category_name,product_category_name_english


In [66]:
# create dictionary
product_dict={keys: values for keys, values in category_name_translation[['product_category_name','product_category_name_english']].values}
product_dict

{'beleza_saude': 'health_beauty',
 'informatica_acessorios': 'computers_accessories',
 'automotivo': 'auto',
 'cama_mesa_banho': 'bed_bath_table',
 'moveis_decoracao': 'furniture_decor',
 'esporte_lazer': 'sports_leisure',
 'perfumaria': 'perfumery',
 'utilidades_domesticas': 'housewares',
 'telefonia': 'telephony',
 'relogios_presentes': 'watches_gifts',
 'alimentos_bebidas': 'food_drink',
 'bebes': 'baby',
 'papelaria': 'stationery',
 'tablets_impressao_imagem': 'tablets_printing_image',
 'brinquedos': 'toys',
 'telefonia_fixa': 'fixed_telephony',
 'ferramentas_jardim': 'garden_tools',
 'fashion_bolsas_e_acessorios': 'fashion_bags_accessories',
 'eletroportateis': 'small_appliances',
 'consoles_games': 'consoles_games',
 'audio': 'audio',
 'fashion_calcados': 'fashion_shoes',
 'cool_stuff': 'cool_stuff',
 'malas_acessorios': 'luggage_accessories',
 'climatizacao': 'air_conditioning',
 'construcao_ferramentas_construcao': 'construction_tools_construction',
 'moveis_cozinha_area_de_ser

### Merging the data sets

In [67]:
brazilian_ecommerce_1 = (clean_orders
        .merge(order_items_dataset, on='order_id', how='inner')        
        .merge(products_dataset, on='product_id', how='inner')         
        .merge(customers_dataset, on='customer_id', how='inner')       
        .merge(order_reviews_dataset, on='order_id', how='inner')  
    )
brazilian_ecommerce_1.head()

,order_id,customer_id,order_status,order_item_id,product_id,product_price,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,1,87285b34884572647811a353c7ac498a,38.71,utilidades_domesticas,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,1,595fac2a385ac33a80bd5114aec74eb8,141.46,perfumaria,400.0,19.0,13.0,19.0,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,8d5266042046a06655c8db133d120ba5,4
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,1,aa4383b373c6aca5d8797843e5594415,179.12,automotivo,420.0,24.0,19.0,21.0,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,e73b67b67587f7644d5bd1a52deb1b01,5
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,1,d0b61bfb1de832b15ba9d266ca96e5b0,72.20,pet_shop,450.0,30.0,10.0,20.0,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,359d03e676b3c069f62cadba8dd3f6e8,5
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,1,65266b2da20d04dbe00c5c2d3bb7859e,28.62,papelaria,250.0,51.0,15.0,15.0,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,e50934924e227544ba8246aeb3770dd4,5


In [68]:
# Now select only your needed columns
brazilian_ecommerce = brazilian_ecommerce_1[['customer_unique_id', 'product_id', 'review_score', 
               'product_category_name', 'product_price', 'customer_state']].drop_duplicates()
brazilian_ecommerce.head()

,customer_unique_id,product_id,review_score,product_category_name,product_price,customer_state
0,7c396fd4830fd04220f754e42b4e5bff,87285b34884572647811a353c7ac498a,4,utilidades_domesticas,38.71,SP
1,af07308b275d755c9edb36a90c618231,595fac2a385ac33a80bd5114aec74eb8,4,perfumaria,141.46,BA
2,3a653a41f6f9fc3d2a113cf8398680e8,aa4383b373c6aca5d8797843e5594415,5,automotivo,179.12,GO
3,7c142cf63193a1473d2e66489a9ae977,d0b61bfb1de832b15ba9d266ca96e5b0,5,pet_shop,72.20,RN
4,72632f0f9dd73dfee390c9b22eb56dd6,65266b2da20d04dbe00c5c2d3bb7859e,5,papelaria,28.62,SP


In [69]:
# rename column
brazilian_ecommerce = brazilian_ecommerce.rename(columns={'review_score': 'ratings'})
brazilian_ecommerce.describe()

,ratings,product_price
count,96700.000000,96700.000000
mean,4.124281,143.880843
std,1.310439,193.335076
min,1.000000,6.080000
25%,4.000000,56.960000
50%,5.000000,94.840000
75%,5.000000,161.502500
max,5.000000,6929.310000


- There are extreme outliers on productPrice

In [70]:
# rename the product_category_name to english
brazilian_ecommerce['product_category_name'] = brazilian_ecommerce['product_category_name'].map(product_dict)
brazilian_ecommerce['product_category_name'].unique()

array(['housewares', 'perfumery', 'auto', 'pet_shop', 'stationery',
       'furniture_decor', 'office_furniture', 'garden_tools',
       'computers_accessories', 'bed_bath_table', 'toys', 'telephony',
       'health_beauty', 'electronics', 'baby', 'cool_stuff',
       'watches_gifts', 'air_conditioning', 'sports_leisure',
       'books_general_interest', 'construction_tools_construction',
       'small_appliances', 'food', 'fashion_underwear_beach',
       'fashion_bags_accessories', 'musical_instruments',
       'luggage_accessories', 'construction_tools_lights',
       'books_technical', 'costruction_tools_garden', 'home_appliances',
       'market_place', 'agro_industry_and_commerce', 'party_supplies',
       'home_confort', 'cds_dvds_musicals', 'consoles_games',
       'furniture_bedroom', 'construction_tools_safety',
       'fixed_telephony', 'drinks',
       'kitchen_dining_laundry_garden_furniture', 'fashion_shoes',
       'home_construction', 'audio', 'home_appliances_2', 'cine

In [71]:
brazilian_ecommerce.drop_duplicates(inplace=True)

In [72]:
brazilian_ecommerce.shape

(96700, 6)

In [73]:
brazilian_ecommerce_clean = brazilian_ecommerce.copy()
brazilian_ecommerce_clean

,customer_unique_id,product_id,ratings,product_category_name,product_price,customer_state
0,7c396fd4830fd04220f754e42b4e5bff,87285b34884572647811a353c7ac498a,4,housewares,38.71,SP
1,af07308b275d755c9edb36a90c618231,595fac2a385ac33a80bd5114aec74eb8,4,perfumery,141.46,BA
2,3a653a41f6f9fc3d2a113cf8398680e8,aa4383b373c6aca5d8797843e5594415,5,auto,179.12,GO
3,7c142cf63193a1473d2e66489a9ae977,d0b61bfb1de832b15ba9d266ca96e5b0,5,pet_shop,72.20,RN
4,72632f0f9dd73dfee390c9b22eb56dd6,65266b2da20d04dbe00c5c2d3bb7859e,5,stationery,28.62,SP
...,...,...,...,...,...,...
106881,6359f309b166b0196dbf7ad2ac62bb5a,ac35486adb7b02598c182c2ff2e05254,5,health_beauty,85.08,SP
106882,da62f9e57a76d978d02ab5362c509660,f1d4ce8c6dd66c47bbaa8c6781c2a923,4,baby,195.00,SP
106883,737520a9aad80b3fbbdad19b66b37b30,b80910977a37536adeddd63663f916ad,5,home_appliances_2,271.01,BA
106884,5097a5312c8b157bb7be58ae360ef43c,d1c427060a0f73f6b889a5c7c61f2ac4,2,computers_accessories,220.58,RJ


In [74]:
brazilian_ecommerce_clean.dtypes

customer_unique_id        object
product_id                object
ratings                    int64
product_category_name     object
product_price            float64
customer_state            object
dtype: object

# Collaborative Filtering (CF)


In [75]:
# required columns for CF
cf_data=brazilian_ecommerce_clean[['customer_unique_id', 'product_id', 'ratings']].drop_duplicates()
cf_data

,customer_unique_id,product_id,ratings
0,7c396fd4830fd04220f754e42b4e5bff,87285b34884572647811a353c7ac498a,4
1,af07308b275d755c9edb36a90c618231,595fac2a385ac33a80bd5114aec74eb8,4
2,3a653a41f6f9fc3d2a113cf8398680e8,aa4383b373c6aca5d8797843e5594415,5
3,7c142cf63193a1473d2e66489a9ae977,d0b61bfb1de832b15ba9d266ca96e5b0,5
4,72632f0f9dd73dfee390c9b22eb56dd6,65266b2da20d04dbe00c5c2d3bb7859e,5
...,...,...,...
106881,6359f309b166b0196dbf7ad2ac62bb5a,ac35486adb7b02598c182c2ff2e05254,5
106882,da62f9e57a76d978d02ab5362c509660,f1d4ce8c6dd66c47bbaa8c6781c2a923,4
106883,737520a9aad80b3fbbdad19b66b37b30,b80910977a37536adeddd63663f916ad,5
106884,5097a5312c8b157bb7be58ae360ef43c,d1c427060a0f73f6b889a5c7c61f2ac4,2


### (a) Memory-Based CF

In [76]:
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [77]:
# encode ids to numeric
user_ids = cf_data['customer_unique_id'].astype('category').cat.codes
item_ids = cf_data['product_id'].astype('category').cat.codes

In [78]:
# build sparse matrix
user_item_matrix = csr_matrix((cf_data['ratings'], (user_ids, item_ids)))

#### - User-based CF - Similar users

In [79]:
# train kNN on users
user_knn = NearestNeighbors(metric='cosine', algorithm='brute')
user_knn.fit(user_item_matrix)

# top-5 similar users for user
distances, indices = user_knn.kneighbors(user_item_matrix[0], n_neighbors=5)
print(indices)

[[23673  6493  5433 25319 64715]]


### - Item-based CF - similar items
- recommend items similar to what the user already liked

In [80]:
# train kNN on items
item_knn = NearestNeighbors(metric='cosine', algorithm='brute')
item_knn.fit(user_item_matrix.T)

# top-5 similar items for product
distances, indices = item_knn.kneighbors(user_item_matrix.T[0], n_neighbors=5)
print(indices)

[[    0 20775 20776 20777 20773]]


#### Memory-based Evaluation
- RMSE or MAE: prediction accuracy, tells how close predicted ratings are to true ratings
- Precision or Recall: recommendation quality, tells how many good recommendations are in the top-K list

In [81]:
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate

# define a Reader with rating scale
reader = Reader(rating_scale=(1, 5))

# load into Surprise dataset
data = Dataset.load_from_df(cf_data[['customer_unique_id', 'product_id', 'ratings']], reader)

# choose similarity-item-based
sim_options = {'name': 'cosine', 'user_based': False}

# build the algorithm
algo = KNNBasic(sim_options=sim_options)

# evaluate
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3061  1.3059  1.3029  1.2945  1.3056  1.3030  0.0044  
MAE (testset)     1.0141  1.0170  1.0129  1.0088  1.0174  1.0140  0.0031  
Fit time          23.61   28.48   31.37   24.47   25.36   26.66   2.87    
Test time         0.52    0.47    0.36    0.47    0.35    0.43    0.07    


{'test_rmse': array([1.30605879, 1.30594266, 1.30291239, 1.29454782, 1.30559958]),
 'test_mae': array([1.01411353, 1.01697216, 1.01293182, 1.00877907, 1.01741115]),
 'fit_time': (23.60695743560791,
  28.48050022125244,
  31.370039463043213,
  24.46851634979248,
  25.36342716217041),
 'test_time': (0.5193850994110107,
  0.4682314395904541,
  0.3632326126098633,
  0.46916842460632324,
  0.3500983715057373)}

### (b) Model-Based Collaborative Filtering using Matrix Factorization (SVD)

In [82]:
from surprise import Dataset, Reader, SVD, accuracy, KNNBasic
from surprise.model_selection import train_test_split

In [83]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(cf_data[['customer_unique_id', 'product_id', 'ratings']], reader)

In [84]:
# split the dataset
trainset, testset = train_test_split(data, test_size=0.2)

In [85]:
# train a basic SVD model- Matrix Factorization
cf_algo = SVD()
cf_algo.fit(trainset);

In [86]:
# model evaluation
predictions = cf_algo.test(testset)
accuracy.rmse(predictions)

RMSE: 1.2991


1.2991465051461772

- RMSE: 1.2744, is a good performance but needs improvement

### SVD hyperparameters tuning using GridSearchCV

In [87]:
from surprise.model_selection import GridSearchCV

param_grid = {
    'n_factors': [50, 100, 150],   # latent factors
    'lr_all': [0.002, 0.005, 0.007], # learning rate
    'reg_all': [0.02, 0.05, 0.1]   # regularization
}

grd_s = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
grd_s.fit(data)

print(grd_s.best_score['rmse'])
print(grd_s.best_params['rmse'])

1.2847648995764545
{'n_factors': 50, 'lr_all': 0.007, 'reg_all': 0.1}


In [88]:
cf_algo = SVD(**grd_s.best_params['rmse'])
trainset = data.build_full_trainset()
cf_algo.fit(trainset)

In [89]:
# model evaluation
predictions = cf_algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8946


0.8946255496964973

- Goog performance

### Recommendations for a specific user

In [90]:
# identify the user
user_id = '7c396fd4830fd04220f754e42b4e5bff'

# list all product_ids
all_products = cf_data['product_id'].unique()

# get products already rated by the user
rated_products = cf_data[cf_data['customer_unique_id'] == user_id]['product_id'].values

In [91]:
# predict ratings for all products not-yet-rated products
recommendations = []
for pid in all_products:
    if pid not in rated_products:
        pred = cf_algo.predict(user_id, pid)
        recommendations.append((pid, pred.est))

recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
# top 5 recommended products
top5 = recommendations[:5]
print(top5)

[('62c89abe1afe3a23c17765d462718a4c', 4.967466594375666), ('3e4176d545618ed02f382a3057de32b4', 4.898084940834559), ('2722b7e5f68e776d18fe901638034e54', 4.879913597390681), ('f6ce2f79a280fa3c1ddc114d34ab273d', 4.852090691787856), ('b9ee7519d0187d2389af62ba6c612963', 4.847142855588542)]


### SVD++ an extension of SVD

In [92]:
from surprise import SVDpp
cf_algo = SVDpp()
cf_algo.fit(trainset)
predictions = cf_algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9179


0.9179310538530837

### (c) Hybrid - Ensemble

In [93]:
# data
ens_data=brazilian_ecommerce_clean[['customer_unique_id', 'product_id', 'ratings', 'product_category_name', 'product_price']].drop_duplicates()

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# copy data
df = ens_data.copy()

In [95]:
# encode user and item for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['customer_unique_id', 'product_id', 'ratings']], reader)

In [96]:
# Split into train-test
trainset = data.build_full_trainset()
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [97]:
# train CF model (SVD)
ens_algo = SVD()
ens_algo.fit(trainset)

# get CF predictions for test_df
cf_preds = []
for _, row in test_df.iterrows():
    pred = ens_algo.predict(row['customer_unique_id'], row['product_id']).est
    cf_preds.append(pred)
test_df['cf_pred'] = cf_preds

In [98]:
# train content-based model

# Encode category
cat_enc = LabelEncoder()
df['cat_idx'] = cat_enc.fit_transform(df['product_category_name'])
test_df['cat_idx'] = cat_enc.transform(test_df['product_category_name'])
train_df['cat_idx'] = cat_enc.transform(train_df['product_category_name'])

In [99]:
# scale price
scaler = MinMaxScaler()
train_df['price_scaled'] = scaler.fit_transform(train_df[['product_price']])
test_df['price_scaled'] = scaler.transform(test_df[['product_price']])

In [100]:
# train regression on content features
content_features = ['cat_idx', 'price_scaled']
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(train_df[content_features], train_df['ratings'])

# predict ratings on test_df
test_df['content_pred'] = rf.predict(test_df[content_features])

In [101]:
# merge predictions_simple average (0.7 CF + 0.3 Content)
alpha = 0.7
test_df['hybrid_pred'] = alpha * test_df['cf_pred'] + (1 - alpha) * test_df['content_pred']

In [102]:
# evaluate
rmse_cf = np.sqrt(mean_squared_error(test_df['ratings'], test_df['cf_pred']))
rmse_content = np.sqrt(mean_squared_error(test_df['ratings'], test_df['content_pred']))
rmse_hybrid = np.sqrt(mean_squared_error(test_df['ratings'], test_df['hybrid_pred']))

print(f"CF RMSE: {rmse_cf:.4f}")
print(f"Content RMSE: {rmse_content:.4f}")
print(f"Hybrid RMSE: {rmse_hybrid:.4f}")

CF RMSE: 0.8607
Content RMSE: 1.4692
Hybrid RMSE: 1.0046


### (d) TensorFlow (NeuMF + content features)
Neural Collaborative Filtering (NCF) using TensorFlow

In [103]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

In [104]:
neuCF_data=brazilian_ecommerce_clean.iloc[:,:-1]
neuCF_data

,customer_unique_id,product_id,ratings,product_category_name,product_price
0,7c396fd4830fd04220f754e42b4e5bff,87285b34884572647811a353c7ac498a,4,housewares,38.71
1,af07308b275d755c9edb36a90c618231,595fac2a385ac33a80bd5114aec74eb8,4,perfumery,141.46
2,3a653a41f6f9fc3d2a113cf8398680e8,aa4383b373c6aca5d8797843e5594415,5,auto,179.12
3,7c142cf63193a1473d2e66489a9ae977,d0b61bfb1de832b15ba9d266ca96e5b0,5,pet_shop,72.20
4,72632f0f9dd73dfee390c9b22eb56dd6,65266b2da20d04dbe00c5c2d3bb7859e,5,stationery,28.62
...,...,...,...,...,...
106881,6359f309b166b0196dbf7ad2ac62bb5a,ac35486adb7b02598c182c2ff2e05254,5,health_beauty,85.08
106882,da62f9e57a76d978d02ab5362c509660,f1d4ce8c6dd66c47bbaa8c6781c2a923,4,baby,195.00
106883,737520a9aad80b3fbbdad19b66b37b30,b80910977a37536adeddd63663f916ad,5,home_appliances_2,271.01
106884,5097a5312c8b157bb7be58ae360ef43c,d1c427060a0f73f6b889a5c7c61f2ac4,2,computers_accessories,220.58


In [105]:
# create function
def train_neuCF(data, mf_dim=32, mlp_layers=[64, 32, 16], epochs=8, batch_size=1024):
    # copy data
    data=neuCF_data.copy()
    # prepare encoders
    user_enc = LabelEncoder()
    item_enc = LabelEncoder()
    cat_enc = LabelEncoder()

    data['user_idx'] = user_enc.fit_transform(data['customer_unique_id'])
    data['item_idx'] = item_enc.fit_transform(data['product_id'])
    data['cat_idx'] = cat_enc.fit_transform(data['product_category_name'])

    # scale price between 0 and 1 
    price_scaler = MinMaxScaler()
    data['price_scaled'] = price_scaler.fit_transform(data[['product_price']])

    # train-test-split
    num_users = data['user_idx'].nunique()
    num_items = data['item_idx'].nunique()
    num_cats = data['cat_idx'].nunique()
    
    train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

    # build hybrid NeuMF model
    user_input = layers.Input(shape=(), dtype='int32', name='user_input')
    item_input = layers.Input(shape=(), dtype='int32', name='item_input')
    cat_input  = layers.Input(shape=(), dtype='int32', name='cat_input')
    price_input = layers.Input(shape=(1,), dtype='float32', name='price_input')

    # embeddings
    user_emb = layers.Embedding(num_users, mf_dim)(user_input)
    item_emb = layers.Embedding(num_items, mf_dim)(item_input)
    cat_emb = layers.Embedding(num_cats, mf_dim//4)(cat_input)
    
    # flatten embeddings
    user_vec = layers.Flatten()(user_emb)
    item_vec = layers.Flatten()(item_emb)
    cat_vec  = layers.Flatten()(cat_emb)
    
    # combine all features
    concat_vec = layers.Concatenate()([user_vec, item_vec, cat_vec, price_input])
    
    #  create dense layers
    x = concat_vec
    for layer_size in mlp_layers:
        x = layers.Dense(layer_size, activation='relu')(x)
    output = layers.Dense(1, activation='linear')(x)

    model = keras.Model(inputs=[user_input, item_input, cat_input, price_input], outputs=output)
    model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

    # train the model
    train_inputs = {'user_input': train_df['user_idx'].values, 'item_input': train_df['item_idx'].values, \
    'cat_input': train_df['cat_idx'].values, 'price_input': train_df[['price_scaled']].values}
    train_targets = train_df['ratings'].astype('float32').values

    val_inputs = {'user_input': test_df['user_idx'].values, 'item_input': test_df['item_idx'].values, \
    'cat_input': test_df['cat_idx'].values, 'price_input': test_df[['price_scaled']].values}
    val_targets = test_df['ratings'].astype('float32').values
    
    model.fit(train_inputs, train_targets, validation_data=(val_inputs, val_targets), batch_size=batch_size, epochs=epochs, verbose=2)

    # predictions and rmse on test
    preds = model.predict(val_inputs, verbose=0).flatten()
    test_rmse = np.sqrt(mean_squared_error(val_targets, preds))

    return model, user_enc, item_enc, test_df, preds, test_rmse

In [106]:
model, user_enc, item_enc, test_df, preds, test_rmse = train_neuCF(neuCF_data, mf_dim=64, mlp_layers=[128, 64, 32], epochs=8)
print("Test RMSE:", test_rmse)

Epoch 1/8
76/76 - 11s - 150ms/step - loss: 6.9618 - root_mean_squared_error: 2.6385 - val_loss: 1.7747 - val_root_mean_squared_error: 1.3322
Epoch 2/8
76/76 - 5s - 63ms/step - loss: 1.2583 - root_mean_squared_error: 1.1217 - val_loss: 1.7767 - val_root_mean_squared_error: 1.3329
Epoch 3/8
76/76 - 5s - 67ms/step - loss: 0.4134 - root_mean_squared_error: 0.6430 - val_loss: 1.8755 - val_root_mean_squared_error: 1.3695
Epoch 4/8
76/76 - 5s - 64ms/step - loss: 0.2220 - root_mean_squared_error: 0.4712 - val_loss: 1.9437 - val_root_mean_squared_error: 1.3942
Epoch 5/8
76/76 - 5s - 63ms/step - loss: 0.1588 - root_mean_squared_error: 0.3985 - val_loss: 1.9274 - val_root_mean_squared_error: 1.3883
Epoch 6/8
76/76 - 5s - 64ms/step - loss: 0.1040 - root_mean_squared_error: 0.3225 - val_loss: 1.9540 - val_root_mean_squared_error: 1.3979
Epoch 7/8
76/76 - 5s - 63ms/step - loss: 0.0790 - root_mean_squared_error: 0.2812 - val_loss: 1.9508 - val_root_mean_squared_error: 1.3967
Epoch 8/8
76/76 - 5s - 64

In [108]:
# copy data
data=neuCF_data.copy()
# prepare encoders
user_enc = LabelEncoder()
item_enc = LabelEncoder()
cat_enc = LabelEncoder()

data['user_idx'] = user_enc.fit_transform(data['customer_unique_id'])
data['item_idx'] = item_enc.fit_transform(data['product_id'])
data['cat_idx'] = cat_enc.fit_transform(data['product_category_name'])

# scale price between 0 and 1 
price_scaler = MinMaxScaler()
data['price_scaled'] = price_scaler.fit_transform(data[['product_price']])

def recommend_user(model, user_enc, user_id, items_df, topN=10):
    user_idx = user_enc.transform([user_id])[0]
    user_array = np.full(len(items_df), user_idx)
    pred_inputs = {'user_input': user_array, 'item_input': items_df['item_idx'].values, 'cat_input': items_df['cat_idx'].values, \
        'price_input': items_df[['price_scaled']].values}
    
    scores = model.predict(pred_inputs, verbose=0).flatten()
    items_df = items_df.copy()
    items_df['pred_score'] = scores
    
    return items_df.sort_values('pred_score', ascending=False).head(topN)

items_df = data[['product_id','item_idx','cat_idx','price_scaled']].drop_duplicates('item_idx')
top10 = recommend_user(model, user_enc, user_id, items_df, topN=10)
print(top10['product_id'].tolist())

['81e61097d1704e63a1d881c02d76934a', '6b5eb8d183cf59e028117fbf4a43b28c', 'cbe952607a2215211502095c631ea179', '6f735de7025b8e74fc832dfd6ec2bf5d', 'b246df143b8670b035cae900adf9654d', '53d090c54ee11e8f6ea8cedfcce57519', 'e83e378e0d9201ebc31bbf3ceb57f9fe', '12f749c76f15e9bf5743bebda92da24a', 'b8fdfcbb0ed03c62d197327a56593842', '377d0ea8bb2c8365af0b1519223d324a']
